#  디스크립터란

프로토콜, 규약

`__get__`, `__set__`, `__delete__` 중 하나가 구현되어 있는 클래스를 디스크립터 클래스

# 1.1  디스크립터를 만들고 내부 인스턴스에 속성 넣기

## 디스크립터 클래스는 정의를 하고 다른 클래스의 클래스 속성에 인스턴스로 처리

## 디스크리터의 값을 디스크립터 속성에 넣도록 구성해서 실제 조회하지 않으면 값을 볼 수 없음

In [1]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    
    def __set__(self, instance, value):
        print("set")
        self.value = value
        
    def __delete__(self, instance):
        print("delete")
        del self.value

## 반드시 클래스 속성으로 정의 

In [2]:
class A:
    value = Descriptor(10) # instance
    value1 = Descriptor(10) # instance

## `__get__`

In [3]:
a = A()

## 디스크립터는 클래스 내부의 속성을 확인하고 이를 이용해서 내부의 메소드를 호출해서 처리

In [4]:
type(a).__dict__['value'].__get__(a, type(a))

get


10

In [5]:
a.value # __get__ 

get


10

## `__set__`

## 디스크립터는 클래스의 속성을 읽어서 갱신도 함

In [6]:
type(a).__dict__['value'].__set__(a, 12)

set


In [7]:
a.value

get


12

In [8]:
a.value1 = 13  

set


In [9]:
a.value1

get


13

### `__delete__`

## 실제 삭제된 거을 디스크리터로 만들어진 객체 

In [10]:
type(a).__dict__['value'].__delete__(a)

delete


In [11]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x504b6a0>,
              'value1': <__main__.Descriptor at 0x504b6d8>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [12]:
try : 
    a.value
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


In [13]:
del a.value1 

delete


In [14]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x504b6a0>,
              'value1': <__main__.Descriptor at 0x504b6d8>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [15]:
try : 
    a.value1
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


## 다시 세팅하면 디스크립터가 작동되어 처리됨 

In [16]:
a.value =123

set


In [17]:
a.value

get


123

In [18]:
type(a).__dict__['value1'].__set__(a,999)

set


In [19]:
a.value1

get


999

# 2.  디스크립터 프로토콜이 없다면..

In [20]:
class D:
    def __getattribute__(self, name):
        print("getattribute")
        return super().__getattribute__(name)


In [21]:
class A:
    value = D()

In [22]:
a = A()
a.value.data = 1

In [23]:
a.value.data

getattribute


1

# 3.  읽기 전용  `__get__` in 디스크립터

In [24]:
class FirstDescriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    

class A:
    a = FirstDescriptor(3)

In [25]:
a = A()
a.a

get


3

## `__set__` 

읽기 전용인것처럼

In [26]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, onwer):
        print("get")
        return self.value
    
#     def __set__(self, instance, value):
#         print("set")
#         self.value = value  # 쓰기 가능
        
    def __set__(self, instance, value):
        # 읽기전용
        print("set")
        raise AttributeError("read only")
        
class A:
    a = Descriptor(3)
    
a = A()

try : 
    a.a = 12
except Exception as e :
    print(e)
a.a

set
read only
get


3

# 4. 디스크립터에서 객체를 저장하는 위치

## 디스크립터에 저장

In [27]:
class Descriptor:
    def __get__(self, instance, owner): # instance = 클래스 A의 인스턴스
        print("get")
        return self.value # self는 디스크립터 클래스의 인스턴스,
    
    def __set__(self, instance, value):
        print('set')
        self.value = value # self는 디스크립터 클래스의 인스턴스,
        
class A:
    x = Descriptor()

In [28]:
a = A()
a.x = 150

set


In [29]:
a.x

get


150

In [30]:
a.__dict__

{}

In [31]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'x': <__main__.Descriptor at 0x52c50b8>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [32]:
b = A()

In [33]:
b.x

get


150

In [34]:
class A:
    x = 3
    
a = A()
b = A()
a.__dict__, b.__dict__

({}, {})

In [35]:
a.x # == a.__dict__['x'] -> type(a).__dict__['x']

3

In [36]:
b.x

3

In [37]:
a.x = 123

In [38]:
a.x

123

In [39]:
type(a).__dict__['x']

3

In [40]:
a.__dict__

{'x': 123}

## 5.  인스턴스에 저장

In [41]:
class descriptor:
    def __get__(self, instane, owner):
        print('get')
        return instance.value
    
    def __set__(self, instance, value):
        print('set')
        instance.value = value
        

class A:
    x = descriptor()

In [42]:
a = A()

In [43]:
a.x = 123

set


In [44]:
b = A()

In [45]:
try :
    b.x
except Exception as e :
    print(e)

get
name 'instance' is not defined


In [46]:
a.__dict__

{'value': 123}

# 6.  무엇을 저장하느냐..

## 함수를 저장

In [47]:
class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("get")
        return self.func(instance) # 메소드는 첫번째 인자로 self 자기자신을 받아야되기 때문에
    

class A:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    
    @descriptor
    def func(self):
        return self.a + self.b + self.c


In [48]:
a = A(1, 2, 3)

In [49]:
a.func # __call__ X

get


6

# 7.  인자를 받도록..

In [50]:
from functools import partial


class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print('get')
        return partial(self.func, instance)
    

class A:
    @descriptor
    def func(self, a, b, c):
        return a + b + c

In [51]:
a = A()

In [52]:
a.func(1, 2, 3)

get


6

# 8. 도전 : 디스크립터로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

### 함수를 저장하는 방식으로

### sample

```
class MutableAttribute:
    __get__ , __set__, __delete__


class ImmutableAttribute:
    __get__, __set__, __delete__ # immutable 에러발생


class Circle:
    radius = ImMutableAttribute()
    pi = ImMutableAttribute()
    
    @MutableAttribute
    def area(self):
        return self.radius ** 2 * self.pi
        
```

# 9 . 디스크립터 메소드 구현시.. 인자는

## `__get__` -> self, instance, owner

## `__set__` -> self, instance, value

## `__delete__` -> self, instance

In [53]:
class MutableAttribute:
    def __init__(self, value=None):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value



In [54]:
class ImmutableAttribute:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("read only")
        
    def __delete__(self, instance):
        raise AttributeError("read only")

In [55]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self : self.radius * 2)
    
#     @ImmutableAttribute
#     def diameter(self):
#         return self.radius * 2
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius **2 * 2

In [56]:
c = Circle()

In [57]:
c.radius

10

In [58]:
c.diameter

20

In [59]:
c.area

200

In [60]:
c.radius = 100

In [61]:
c.area

20000

In [62]:
try :
    c.area = 123
except Exception as e :
    print(e)

read only


# 10. 도전 : 실행가능 디스크립터


```
class A:
    @descriptor
    def sum(self, a, b, c):
        return a + b + c
        
a = A()
a.sum(1, 2, 3) # 6
```

+ partial 객체를 이용 (functools.partial)

In [63]:
from functools import partial

class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        # return self.func(instance)
        return partial(self.func, instance)



In [64]:
class A:
    @descriptor
    def sum(self, a, b, c):
        return a + b + c

In [65]:
a = A()

In [66]:
a.sum

functools.partial(<function A.sum at 0x000000000508E0D0>, <__main__.A object at 0x0000000005068AC8>)

In [67]:
a.sum(1, 2, 3) # 6

6

In [68]:
type(a).__dict__['sum'].__get__(a, type(a))(1, 2, 3)

6

In [69]:
class A:
    def sum(self):
        pass

In [70]:
a = A()
type(a).__dict__['sum'].__get__

<method-wrapper '__get__' of function object at 0x0000000005059F28>